In [28]:
import pickle
import numpy as np
import pandas as pd
import scipy.stats as st
from numba import njit, prange
from math import sin, cos, sqrt, atan2, radians

In [29]:
path = '../data/'

In [30]:
!ls $path

D_p-pts.pickle	exp1-trj.label	porto.csv     train.src  val.trg
D_q-pts.pickle	exp1-trj.pts	README.md     train.trg  vocab.txt
Dq-pts.pickle	exp1-trj.t	saved_models  val.src


In [31]:
# Reloading Dq...
with open(path+'Dq-pts.pickle', 'rb') as file:
    Dq = pickle.load(file)

# Reloading D_q...
with open(path+'D_q-pts.pickle', 'rb') as file:
    D_q = pickle.load(file)

# Reloading D_p...
with open(path+'D_p-pts.pickle', 'rb') as file:
    D_p = pickle.load(file)

In [38]:
Dq[0]

array([[-8.585676, 41.148621],
       [-8.585721, 41.148864],
       [-8.587161, 41.149044],
       [-8.590383, 41.150052],
       [-8.593281, 41.150772],
       [-8.593254, 41.150808],
       [-8.59401 , 41.150493],
       [-8.593335, 41.148324],
       [-8.593056, 41.147415],
       [-8.592993, 41.147397],
       [-8.592939, 41.147352],
       [-8.59293 , 41.147334]])

In [35]:
D_q[0]

array([[-8.585694, 41.148639],
       [-8.586135, 41.14899 ],
       [-8.588574, 41.149557],
       [-8.592345, 41.150547],
       [-8.593272, 41.150808],
       [-8.59356 , 41.150871],
       [-8.593767, 41.149683],
       [-8.593083, 41.14746 ],
       [-8.59302 , 41.147406],
       [-8.592966, 41.147379],
       [-8.59293 , 41.147343],
       [-8.59293 , 41.147325]])

In [51]:
D_p[0]

array([[-8.620002, 41.147901],
       [-8.620308, 41.147343],
       [-8.620146, 41.146308],
       [-8.620155, 41.14629 ],
       [-8.618013, 41.145975],
       [-8.616114, 41.145246],
       [-8.613639, 41.145894],
       [-8.611974, 41.146002],
       [-8.61093 , 41.145723],
       [-8.61102 , 41.145021],
       [-8.612577, 41.143869],
       [-8.613864, 41.143023],
       [-8.615214, 41.142087],
       [-8.615097, 41.140836],
       [-8.613495, 41.141286]])

In [52]:
@njit
def dist_haversine(p1, p2):
    """
    p1 --> numpy array [lon, lat]
    p2 --> numpy array [lon, lat]
    """
    # approximate radius of earth in km
    R = 6371.0

    lon1 = radians(p1[0]) # p1[x, ]
    lat1 = radians(p1[1]) # p1[ ,y]
    lon2 = radians(p2[0]) # p2[x, ]
    lat2 = radians(p2[1]) # p2[ ,y]

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance * 1000

In [161]:
@njit
def dist_lcss(T1, T2): # Programação Dinâmica da LCSS para trajetórias
    m = len(T1) # length of the traj T1
    n = len(T2) # length of the traj T2
    thr = 225 # thr --> threshold in meters

    # Declarando a matrix L(m+1 x n+1), lista de listas!
    L = [[0]*(n + 1) for _ in range(m + 1)]

    for i in prange(m + 1):
        for j in prange(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif dist_haversine(T1[i-1], T2[j-1]) <= thr:
                L[i][j] = L[i-1][j-1] + 1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])

    lcss_length = L[m][n]
    
    return 1 - (lcss_length/max(m, n))

In [162]:
t1 =  np.array([[-37.97151, -4.93422],
                [-37.971341, -4.934051],
                [-37.971111, -4.933850],
                [-37.970728, -4.933544],
                [-37.970429, -4.933256],
                [-37.970207, -4.933063]], dtype=np.float32)

t2 =  np.array([[-37.97135, -4.93438],
                [-37.971143, -4.934198],
                [-37.970769, -4.933945],
                [-37.969606, -4.934348]], dtype=np.float32)

In [163]:
dist_lcss(t1, t2)

0.33333333333333337

In [164]:
dist_lcss(t1, t1)

0.0

In [167]:
def rank(t_query, i, db_search):
    dists = list(map(lambda t: dist_lcss(t_query, t), db_search)) # pega todas as distâncias de Dq[i] as trajs de D_qUD_p
    dists = np.array(dists)
    order = dists.argsort() # pega a ordem
    ranks = order.argsort() # pega o rank

    return ranks[i] + 1 # retorna o ranking de ta' no dbsearch | soma +1 pq o argsort rankea a partir de 0 (zero)

In [168]:
def acc(ranks):
    count = 0
    for i in range(len(ranks)):
        if ranks[i] == 1:
            count += 1

    return round(count/len(ranks), 3)

In [169]:
def mr(ranks):
    return (sum(ranks)/len(ranks))

In [170]:
def mrr(ranks):
    count = 0
    for i in range(len(ranks)):
        count += 1/ranks[i]

    return round(count/len(ranks), 3)

In [171]:
# Intervalo de Confiança do Ranks
def cip_r(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(ranks[i]) # Add os Ranks
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [172]:
# Intervalo de Confiança dos Reciprocal Ranks
def cip_rr(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(1/ranks[i]) # Add os RRs...
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [173]:
%%time
dbsizes = [20000, 40000, 60000, 80000, 100000]
entire_db = D_q + D_p
for dbsize in dbsizes:
    ranks = []
    dbsearch = entire_db[:dbsize]
    for i in range(len(Dq)):
        ranks.append(rank(Dq[i], i, dbsearch)) # rank de ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 31.196(-5.704, 68.096), Acc: 0.788, MRR: 0.849(0.83, 0.867) with dbsearch size: 20000
Mean rank: 62.754(-13.522, 139.03), Acc: 0.741, MRR: 0.812(0.791, 0.832) with dbsearch size: 40000
Mean rank: 92.831(-20.312, 205.974), Acc: 0.713, MRR: 0.787(0.765, 0.808) with dbsearch size: 60000
Mean rank: 123.373(-26.742, 273.488), Acc: 0.687, MRR: 0.766(0.744, 0.789) with dbsearch size: 80000
Mean rank: 155.148(-33.647, 343.943), Acc: 0.677, MRR: 0.753(0.73, 0.776) with dbsearch size: 100000
CPU times: user 2h 41min 31s, sys: 116 ms, total: 2h 41min 31s
Wall time: 2h 41min 31s
